# Пишем сайты на питоне

Оригинальные [конспект1](https://github.com/elmiram/2016learnpython/blob/master/7%20%D0%A1%D0%B5%D0%BC%D0%B8%D0%BD%D0%B0%D1%80%20-%20flask%20intro.ipynb) [конспект2](https://github.com/elmiram/2016learnpython/blob/master/8%20%D0%A1%D0%B5%D0%BC%D0%B8%D0%BD%D0%B0%D1%80%20-%20%D0%A1%D0%BD%D0%BE%D0%B2%D0%B0%20flask.ipynb)

Написать сайт на питоне значит написать такую программу, которая может работать веб-сервером или использоваться веб-сервером для порождения HTML-кода веб-страниц. Для этого существует несколько модулей, например, Django и Flask

### Flask: часть 1

* [Установка Flask](https://flask.palletsprojects.com/en/1.1.x/installation/)

* Программа, написанная с помощью flask, работает, как веб-сервер.
* За каждую веб-страницу сайта отвечает какая-то функция, которая возвращает HTML-код этой страницы. 
* Естественно, писать длинные куски HTML-кода внутри программы на питоне было бы странно, поэтому функции могут загружать HTML из заранее заготовленных файлов.
* Эти файлы могу содержать готовые страницы или шаблоны страниц на специальном языке.

##### Структура проекта

```
project
│─── myapp.py
│─── file001.txt    
└─── templates
│    │─── index.html
│    └─── ...   
└─── static
     │─── css
     │─── js
     └─── images
```
- **myapp.py** $-$ основной файл, в котором находится python-код приложения

- папка **templates** $-$ в ней лежат [HTML-шаблоны](https://www.w3schools.com/html/) для страниц, которые будут на сайте
    
- папка **static** $-$ в ней лежат файлы со [стилями css](https://github.com/hse-ling-python/seminars/blob/master/html_and_requests/html_css.ipynb), скрипты, изображения


**[Пример проекта сайта, подключённого к imdb-БД](https://github.com/hse-ling-python/imdb-site-example)**

____________

### Напишем простой сайт на flask:

In [1]:
from flask import Flask

app = Flask(__name__)

@app.route('/') 
def index():
    return '<html><body><p>Hello, world!</p></body></html>'

if __name__ == '__main__': # сервер запустится только если этот код не импортируется другим скриптом
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [01/Dec/2025 12:48:28] "GET / HTTP/1.1" 200 -


Если требуется, вы можете задать конкретный хост и порт, на котором вы запускаете сайт (бывает полезно, если у вас много разного на разных портах):

In [3]:
if __name__ == '__main__':
    app.run(debug=False, host='0.0.0.0', port=5001)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://192.168.0.100:5001
Press CTRL+C to quit
127.0.0.1 - - [01/Dec/2025 12:51:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2025 12:51:13] "GET /favicon.ico HTTP/1.1" 404 -
192.168.0.100 - - [01/Dec/2025 12:51:17] "GET / HTTP/1.1" 200 -


Почему Running on... и два адреса? Это один и тот же сервер на порту 5001, просто к нему можно подключиться двумя способами:

с этого же устройства (компьютера): http://127.0.0.1:5001 или http://localhost:5001

с других устройств в той же сети: http://192.168.0.100:5001

### Страницы сайта

Каждая страница сайта порождается какой-то функцией. Декоратор `@app.route(...)` перед функцией показывает, какой адрес будет у страницы, за которую отвечает эта функция:

In [ ]:
app = Flask(__name__)

@app.route('/') # внутренний путь на страничку. Здесь путь не прописан, значит, это главная страница.
def index():
    return 'Main page'

@app.route('/hi')
def hi():
    return 'Hi!'

if __name__ == '__main__': 
    app.run(debug=False)

Одна функция может отвечать за несколько разных страниц:
* во-первых, декораторов может стоять несколько подряд,
* во-вторых, декораторы могут содержать переменные.

Пример декоратора с переменными:

In [10]:
@app.route('/user/<user>')
def user_index(user):
    return 'This is the page of' + user

Посмотрим:

In [ ]:
if __name__ == '__main__':
    app.run(debug=False)

Переменные в адресах могут быть разного типа: `int` — целое число, `float` — действительное число, `path` — строка, которая может содержать слэши. Тип переменной можно указать вот так:

In [ ]:
import datetime

@app.route('/time/<int:shift>')
def time_page(shift):
    h = datetime.datetime.today().hour
    h += shift # разница с GMT
    return 'Time in your country:' + str(h)

### Перенаправление

Для того, чтобы направить пользователя на другую страницу, используется функция `redirect`. Например, вот приложение, в котором есть страница /time. С помощью функции `redirect` можно, например, реализовать такое:

- если пользователь заходит на страницу /time в рабочее время (с 10 до 18), то он перенаправляется на главную страницу сайта,
- если пользователь заходит на страницу в другое время, то он перенаправляется на страницу /come_later.

Страницы часто содержат ссылки друг на друга, но чтобы поставить ссылку, нужно знать адрес. Чтобы узнать адрес страницы, которую создаёт какая-то функция, используется функция url_for:

In [ ]:
from flask import url_for, redirect

app = Flask(__name__)


@app.route('/')
def index():
    return '<html><body><h1>Привет, готов к работе!</h1></body></html>'

#пример множественных декораторов
@app.route('/come_later')
@app.route('/come_later/<user>')
def hi(user=None):
    if user is None:
        user = 'friend'
    return '<html><body><p>Привет, ' + user + '!</p></body></html>'


@app.route('/time')
def time_redirect(shift=-5):
    h = datetime.datetime.today().hour
    h += shift
    if 10 <= h < 18:
        return redirect(url_for('index'))
    return redirect(url_for('hi', user='ночной_гость'))


if __name__ == '__main__':
    app.run(debug=False, host='0.0.0.0', port=5001)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://192.168.0.100:5001
Press CTRL+C to quit
127.0.0.1 - - [01/Dec/2025 13:11:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2025 13:11:50] "GET /ешьу HTTP/1.1" 404 -
127.0.0.1 - - [01/Dec/2025 13:11:54] "GET /time HTTP/1.1" 302 -
127.0.0.1 - - [01/Dec/2025 13:11:54] "GET /come_later/ночной_гость HTTP/1.1" 200 -



### Шаблоны

Динамические веб-страницы собираются из шаблонов: у шаблона есть постоянная часть и изменяющаяся часть. Flask позволяет хранить в файлах такие шаблоны страниц. __Все шаблоны страниц должны находиться в папке `templates`.__

Шаблон состоит из обычного html-кода. Но в этот обычный html могут быть вставлены специальные фрагменты кода, способного порождать разный html в зависимости от значений переменных. 

Эти специальные фрагменты пишутся внутри `{% ... %}` или `{{ ... }}`. При порождении HTML эти фрагменты заменяются на какой-то HTML-код по определённым правилам.

Чтобы породить HTML по шаблону, используется функция `render_template()`:

```
from flask import render_template
@app.route('/hello/')
@app.route('/hello/<name>')
def hello(name=None):
    return render_template('hello.html', name=name)
```

Язык, на котором пишутся эти вставки, похож на питон, __но многие питоновские функции в шаблонах работать не будут!__

__Вставки `{{ ... }}`__

* `{{ ... }}` означает «вычислить значение выражения в скобках и заменить им вставку»
* в скобках может быть переменная, например, `{{ name }}`
* к переменным в скобках __нельзя__ применять произвольные функции: `{{ f(name) }}`
* существует набор встроенных операций, которые можно выполнять над переменными
* эти операции отделяются от переменной знаком | и тогда выражение пишется так `{{ name|length }}`, примеры операций: 
    * `length`
    * `lower`, `upper`
    * `e` — автоматически экранировать спецсимволы HTML
    * `random` — взять случайный элемент из массива
    * `urlencode` — закодировать строку с помощью percent encoding. _Это метод кодирования символов в URL путем замены их на шестнадцатеричный код, предваряемый знаком процента (%). Например: символ пробела кодируется как %20, символ # кодируется как %23, Слово «Лук» на русском языке (в кодировке UTF-8) будет выглядеть как %D0%9B%D1%83%D0%BA._

__Вставки `{% ... %}`__

В таких `{% ... %}` скобках пишутся специальные команды, аналогичные питоновским `if` и `for`.
Примеры:

`{% for i in arr %} ... {% endfor %}`

`{% if ... %} ... {% endif %}`

`{% elif ... %}`

`{% else %}`


Пример фрагмента HTML-кода с условием:
```
{% if username|length > 20 %}
    <p>Слишком длинное имя!</p>
{% else %}
    <p>{{ username }}</p>
{% endif %}
```

## Словари в шаблонах

Предположим, у нас есть словарь, в котором хранятся имена наших друзей и их почтовые адреса. И мы хотим выводить эти имена с адресами на нашем сайте. Мы можем написать что-то такое:


In [ ]:
from flask import Flask
from flask import render_template

app = Flask(__name__)


@app.route('/')
def index():
    emailbook = {'Петя': 'petya@example.com',
                 'Вася': 'vasya@example.com',
                 'Катя': 'katya@example.com'}
    return render_template('index.html', emails=emailbook)

if __name__ == '__main__':
    app.run(debug=False, host='0.0.0.0', port=5001)

Тогда в папке `templates` нам нужно создать файл `index.html`, в котором будут перебираться элементы словаря. Делается это с помощью функции `items()`. Вот так будет выглядеть `index.html`:

```
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Почтовые адреса</title>
</head>
<body>
<h1>Адреса моих друзей</h1>
<ul>
    {% for name, email in emails.items() %}
        <li>{{ name }} - {{ email }}</li>
    {% endfor %}
</ul>
</body>
</html>
```

Когда вы пишете сайт на фласке у себя на компьютере, можно написать вместо `app.run()` вот так: `app.run(debug=True)`. Это значит, что если на сайте возникнет ошибка, то на странице с ошибкой выведется ее описание, и вы легко сможете понять, в каком месте кода все падает. Но когда вы будете выкладывать сайт в Интернет, нужно убедиться, что `debug` отключен.

## Как вывести на веб-страничку содержимое текстового файла?

[Данные](https://github.com/hse-ling-python/seminars/tree/master/flask_applications/flask1_2024_data) для запуска кода ниже.

In [2]:
from flask import Flask, render_template

app = Flask(__name__)

@app.route('/poem')
def poem():
    with open("poem.txt", "r", encoding='utf-8') as f:
        content = f.read().split('\n')
    return render_template("poem.html", content=content)

if __name__ == '__main__':
    app.run(debug=False, host='0.0.0.0', port=5001)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://192.168.0.100:5001
Press CTRL+C to quit


#### Практика:

Создайте сайт, где 

+ на главной странице будет выводиться Ваше любимое стихотворение,
+ с главной страницы можно перейти на страницы stats и thanks,
+ на странице stats выведен частотный словарь стихотворения и соответствующий график.